In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#This imports the necessary path to draw the single pad transmon design. 
import sys
sys.path.append('../Customized_Components')
from single_pad_transmon_pocket import TransmonPocket_Single

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [178]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed, RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

import pyEPR as epr

import Default_res_params as dp

In [24]:
import pandas as pd
import numpy as np

In [4]:
# design = designs.DesignPlanar({}, True)
# design.chips.main.size['size_x'] = '10 mm'
# design.chips.main.size['size_y'] = '10 mm'

# design.variables['cpw_width'] = '15 um'
# design.variables['cpw_gap'] = '9 um'

# gui = MetalGUI(design)

In [5]:
gui = MetalGUI(dp.design)
design = dp.design

In [192]:
coupling_d = [100,105,140,200,2,15,40,75]
curve_d = [380,380,380,400,300,310,330,340]
ground_plane_d = 10
fillets = [200,200,280,320,200,200,200,240]

prime_pin = 14.18
prime_gap = 8

In [45]:
resonator_params = pd.read_csv('results.csv')
print(resonator_params)

   Unnamed: 0  Frequency          Q  Length  Gap    Pin  coupling_gap
0           0        5.9  1278612.0  5.0535   10  17.73           118
1           1        6.2  1210136.0  4.8072   12  21.27           123
2           2        6.5  1149617.0  4.5836   14  24.82           158
3           3        6.7  1113073.0  4.4457   16  28.37           158
4           4        7.0  1063157.0  4.2536    2   3.55            20
5           5        7.2  1032764.0  4.1344    4   7.09            33
6           6        7.5   990927.0  3.9674    6  10.64            58
7           7        7.9   941290.0  3.7646    8  14.18            93


In [56]:
freqs = resonator_params['Frequency'].values
Qs = resonator_params['Q'].values
gaps = resonator_params['Gap'].values
pins = resonator_params['Pin'].values
coupling_gap = resonator_params['coupling_gap'].values
lengths = resonator_params['Length'].values

In [184]:
design.delete_all_components()

In [7]:
# design.delete_component('cpw_2')

In [185]:
design.overwrite_enabled = True #This makes it possible to edit the designs later on. 



# options = dict(pad_width = '250 um',
# pocket_width = '300 um',
# pad_pocket_distance_top = '60 um',
# connection_pads = dict(
#     a = dict(loc_W  = 1, 
#              loc_H =  1, 
#              pad_gap = '10 um',
#              pad_height = '20 um',
#              pad_width = '100 um',
#              pad_cpw_extent =  '10 um',
#              pocket_rise = '0 um',
#              cpw_extend = '5 um')))

# q1 = TransmonPocket_Single(design,'Q1',options = dict(pos_x = '1 mm', 
#                                                       pos_y = '0',
#                                                       pad_height = '200 um', 
#                                                       **options))
# q2 = TransmonPocket_Single(design,'Q2',options = dict(pos_x = '0.39 mm', 
#                                                       pos_y = '-0.7', 
#                                                       pad_height = '220 um', 
#                                                       **options))

# gui.rebuild()
# gui.zoom_on_components(['Q1','Q2'])

In [171]:
# design.delete_all_components()

# prime_width='10um',
#                            prime_gap='6um',
#                            second_width='10um',
#                            second_gap='6um',
#                            coupling_space='3um',
#                            coupling_length='100um',
#                            down_length='100um',
#                            fillet='25um',
#                            mirror=False,
#                            open_termination=True)

# TQ_options = dict(prime_width = design.variables['cpw_width'],
#                prime_gap = design.variables['cpw_gap'],
#                second_width = design.variables['cpw_width'],
#                second_gap = design.variables['cpw_gap'],
#                downlength = '30um',
#                coupling_space = '5um',
#                open_termination=False)

# TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='1.35 mm',
#                                              pos_y='0.5 mm',
#                                              coupling_length='120 um', **TQ_options))
# TQ2 = CoupledLineTee(design, 'TQ2', options=dict(pos_x='0.15 mm',
#                                              pos_y='0.5 mm',
#                                              coupling_length='120 um', mirror = True, **TQ_options))

# gui.rebuild()
# gui.autoscale()

In [180]:
# -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.core import QComponent
import numpy as np


class CoupledLineTee(QComponent):
    """Generates a three pin (+) structure comprised of a primary two pin CPW
    transmission line, and a secondary one pin neighboring CPW transmission
    line that is capacitively/inductively coupled to the primary. Such a
    structure can be used, as an example, for generating CPW resonator hangars
    off of a transmission line.

    Inherits QComponent class.

    ::

        +----------------------------+
        ------------------------------
        |
        |
        |
        |
        +

    .. image::
        CoupledLineTee.png

    .. meta::
        Coupled Line Tee

    Default Options:
        * prime_width: '10um' -- The width of the trace of the two pin CPW transmission line
        * prime_gap: '6um' -- The dielectric gap of the two pin CPW transmission line
        * second_width: '10um' -- The width of the trace of the one pin CPW transmission line
        * second_gap: '6um' -- The dielectric gap of the one pin CPW transmission line
        * coupling_space: '3um' -- The amount of ground plane between the two transmission lines
        * coupling_length: '100um' -- The length of parallel between the two transmission lines
          note: this includes the distance of the curved second of the second line
        * down_length: '100um' -- The length of the hanging part of the resonator, including the curved region
        * fillet: '25um'
        * mirror: False -- Flips the hanger around the y-axis
        * open_termination: True -- sets if the termination of the second line at the coupling side
          is an open to ground or short to ground
    """
    component_metadata = Dict(short_name='cpw', _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(prime_width='10um',
                           prime_gap='6um',
                           prime_length = '120um',
                           second_width='10um',
                           second_gap='6um',
                           coupling_space='3um',
                           pin_drop = '0um',
                           coupling_length='100um',
                           down_length='100um',
                           fillet='25um',
                           mirror=False,
                           open_termination=True)
    """Default connector options"""

    TOOLTIP = """Generates a three pin (+) 
    structure comprised of a primary two 
    pin CPW transmission line, and a 
    secondary one pin neighboring CPW 
    transmission line that is 
    capacitively/inductively coupled 
    to the primary."""

    def make(self):
        """Build the component."""
        p = self.p

        prime_cpw_length = p.prime_length
        second_flip = 1
        if p.mirror:
            second_flip = -1

        #Primary CPW
        prime_cpw = draw.LineString([[-prime_cpw_length / 2, 0],
                                     [prime_cpw_length / 2, 0]])
        
        second_termination = 0
        if p.open_termination:
            second_termination = p.second_gap

        #Secondary CPW
        second_down_length = p.down_length
        second_y = -p.prime_width / 2 - p.prime_gap  - p.second_gap - p.coupling_space
        second_cpw = draw.LineString(
            [[0, second_y-p.pin_drop],
             [0,second_y - second_down_length]])

        

        second_cpw_etch = draw.LineString(
            [[0, second_y + second_termination],
             [0,second_y - second_down_length]])

        #Rotate and Translate
        c_items = [prime_cpw, second_cpw, second_cpw_etch]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [prime_cpw, second_cpw, second_cpw_etch] = c_items

        #Add to qgeometry tables
        self.add_qgeometry('path', {'prime_cpw': prime_cpw},
                           width=p.prime_width)
        self.add_qgeometry('path', {'prime_cpw_sub': prime_cpw},
                           width=p.prime_width + 2 * p.prime_gap,
                           subtract=True)
        self.add_qgeometry('path', {'second_cpw': second_cpw},
                           width=p.second_width,
                           fillet=p.fillet)
        self.add_qgeometry('path', {'second_cpw_sub': second_cpw_etch},
                           width=p.second_width + 2 * p.second_gap,
                           subtract=True,
                           fillet=p.fillet)

        #Add pins
        prime_pin_list = prime_cpw.coords
        second_pin_list = second_cpw.coords

        self.add_pin('prime_start',
                     points=np.array(prime_pin_list[::-1]),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('prime_end',
                     points=np.array(prime_pin_list),
                     width=p.prime_width,
                     input_as_norm=True)
        self.add_pin('second_end',
                     points=np.array(second_pin_list),
                     width=p.second_width,
                     input_as_norm=True)


In [181]:
i = 5

In [194]:

curve_d_i = curve_d[i]
coupling_d_i = coupling_d[i]
pins_i = pins[i]
gap_i = gaps[i]

down_len = -coupling_d_i - ground_plane_d - prime_pin/2 - prime_gap + curve_d_i*np.pi/2


TQ_options = dp.TQ_options
TQ_options['prime_length'] = '120um'
TQ_options['down_length'] =  str(down_len) + 'um'
TQ_options['coupling_space'] = str(ground_plane_d) + 'um'
TQ_options['second_width'] = str(pins[i]) + 'um'
TQ_options['second_gap'] = str(gaps[i]) + 'um'
TQ_options['coupling_space'] = str(coupling_gap[i])+ 'um'
TQ_options['pin_drop'] = str(coupling_d[i])+'um'

TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='0 mm',
                                             pos_y='0 mm',
                                             coupling_length='120 um', **TQ_options))

gui.rebuild()
gui.autoscale()

06:20PM 39s ERROR [rebuild]: ERROR in building component name=cpw, error=ParseException: Expected number but encountered '('


GEOSException: ParseException: Expected number but encountered '('

In [188]:
short = ShortToGround(design, 'short', options=dict(pos_x='0.6 mm', pos_y='-0.5 mm', orientation='270'))

In [189]:
lengths[i]

4.1344

In [ ]:
def calculate_anchors(i,x0,y0,angle = 0, flip = False, down_straight = 250):
    factor = 1
    if flip:
        factor = -1
    fillet = fillets[i]
    length = lengths[i]
    d = curve_d[i]
    length -= down_len
    staight_len = length-2*np.pi*d
    x = staight_len/3
    x1 = x0+(d+x)*np.cos(angle)
    y1 = y0-d/2*np.sin(angle)*factor
    
    x2 = x1 + x*np.cos(angle)
    
    

In [190]:
CPW_options = dp.CPW_options
CPW_options['pin_width'] = str(pins[i]) + 'um'
CPW_options['gap_width'] = str(gaps[i]) + 'um'
CPW_options['total_length'] = str(lengths[i]-250) + 'um'
CPW_options['pin_inputs'] = in_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='second_end'),
            end_pin=Dict(component='short', pin='short'),)

CPW_options['trace_gap'] = str(gaps[i]) + 'um'
CPW_options['trace_width'] = str(pins[i]) + 'um'
CPW_options['fillet'] = '10 um'
CPW_options['lead'] = dict(start_straight = '100 um', end_straight = '100 um')


cpw = RouteMeander(design, 'cpw',options = CPW_options)

gui.rebuild()
gui.autoscale()

06:02PM 57s ERROR [rebuild]: ERROR in building component name=cpw, error=ParseException: Expected number but encountered '('


GEOSException: ParseException: Expected number but encountered '('